In [1]:
from tqdm.notebook import tqdm
from fetcher import fetch_difumo
import numpy as np
from nilearn.maskers import NiftiMapsMasker
import dask.array as da
from dask import delayed
import nibabel as nib
from dask.distributed import Client, LocalCluster

In [2]:
difumo_dim = 64
maps_img = fetch_difumo(dimension=difumo_dim).maps

In [3]:
subjects_fmri = []

for sub in range(1, 101):
    try: 
        subjects_fmri.append(nib.load(f'/home/jovyan/shared/ds003097/derivatives/fmriprep/sub-{sub:04}/func/sub-{sub:04}_task-moviewatching_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'))
    except:
        print(f'Error for {sub}')


Error for 9
Error for 10
Error for 11
Error for 12
Error for 13
Error for 14
Error for 15
Error for 16
Error for 17
Error for 18
Error for 19
Error for 20
Error for 21
Error for 22
Error for 23
Error for 24
Error for 25
Error for 26
Error for 27
Error for 28
Error for 29
Error for 30
Error for 31
Error for 32
Error for 33
Error for 34
Error for 35
Error for 36
Error for 37
Error for 38
Error for 39
Error for 40
Error for 41
Error for 42
Error for 43
Error for 44
Error for 45
Error for 46
Error for 47
Error for 48
Error for 49
Error for 50
Error for 51
Error for 52
Error for 53
Error for 54
Error for 55
Error for 56
Error for 57
Error for 58
Error for 59
Error for 60
Error for 61
Error for 62
Error for 63
Error for 64
Error for 65
Error for 66
Error for 67
Error for 68
Error for 69
Error for 70
Error for 71
Error for 72
Error for 73
Error for 74
Error for 75
Error for 76
Error for 77
Error for 78
Error for 79
Error for 80
Error for 81
Error for 82
Error for 83
Error for 84
Error for 85


In [5]:
subjects_fmri

In [6]:
num_subjects_data = len(subjects_fmri)
num_subjects_data

8

In [9]:
def no_parallel():
    num_frames = 100
    signals_list = []
    
    maps_masker = NiftiMapsMasker(maps_img=maps_img, verbose=0, standardize=True)
    for time in tqdm(range(num_frames), desc='Time'):

        all_subs_time = []
        for sub_fmri in subjects_fmri:
            time_slice = sub_fmri.slicer[:, :, :, time]
            all_subs_time.append(time_slice)

        signals = maps_masker.fit_transform(all_subs_time)
        signals_list.append(signals)

    all_signals = np.array(signals_list)
    return all_signals


In [ ]:
no_parallel()

Time:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
def parallel_work():
    cluster = LocalCluster(n_workers=2, )
    client = Client(cluster)
    
    num_frames = 290
    signals_list_delayed = []

    def process_time_slice(frame_num):
        maps_masker = NiftiMapsMasker(maps_img=maps_img, verbose=0, standardize=True)
        all_subs_frame = []
        for sub_fmri in subjects_fmri:
            time_slice = sub_fmri.slicer[:, :, :, frame_num]
            all_subs_frame.append(time_slice)
        signals = maps_masker.fit_transform(all_subs_frame)
        return signals

    process_time_slice_delayed = delayed(process_time_slice)

    for frame_n in range(num_frames):
        signals_list_delayed.append(
            da.from_delayed(
                process_time_slice_delayed(frame_n), 
                (num_subjects_data, difumo_dim), 
                dtype=float
            )
        )

    all_signals = da.stack(signals_list_delayed, 0)
    # all_signals.visualize(filename='all_signals.svg')
    # result = all_signals.compute()
    my_result = client.compute(all_signals)
    result = client.gather(my_result)
    return result
